# LangChain Expression Language (LCEL)

In this notebook, we will give a high level introduction to what **LangChain Expression Language (LCEL)** is and show we can use it with an MLX locally-deployed model.

## Notebook Setup
Throughout this notebook, we will largely be making use of LangChain alongside MLX. In order to do a direct comparison with how MLX works within LangChain, we will also provide an example using the standard OpenAI API.

In [45]:
# Importing the necessary Python libraries
import yaml
from mlx_lm import load, generate
from typing import Any, List, Mapping, Optional
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.language_models.llms import LLM
from langchain_openai import ChatOpenAI

# Importing legacy LangChain things for demonstration purposes
from langchain.chains import ConversationChain

In [46]:
# Setting constant values to represent model name and directory
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
BASE_DIRECTORY = '../models'
MLX_DIRECTORY = f'{BASE_DIRECTORY}/mlx'
mlx_model_directory = f'{MLX_DIRECTORY}/{MODEL_NAME}'

In [5]:
# Loading my personal OpenAI API key (NOT pushed up to GitHub)
with open('../sensitive/api-keys.yaml') as f:
    API_KEYS = yaml.safe_load(f)

## Chaining the Old Way
In the cell below, we will demonstrate the former way of chaining a prompt template together with an LLM. LangChain has offered multiple ways to chain prompts to LLMs, but perhaps one of the most popular ways was using the `ConversationChain`. Let's go ahead and set up a simple prompt template and chain that to the OpenAI API to demonstrate how this legacy option worked.

In [14]:
# Instantiating the OpenAI LLM
llm = ChatOpenAI(openai_api_key = API_KEYS['OPENAI_API_KEY'])

In [19]:
# Setting up the Chat prompt template
system_message_prompt = SystemMessagePromptTemplate.from_template(template = 'You are a helpful assistant.')
human_message_prompt = HumanMessagePromptTemplate.from_template(template = "History: {history}\nHuman: {input}")
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [20]:
# Instantiating the legacy conversation chain
legacy_conversation_chain = ConversationChain(llm = llm, prompt = chat_prompt)

In [24]:
# Making a call to the OpenAI API
response = legacy_conversation_chain.predict(input = 'Write me a haiku about flowers.')

print(response)

Blossoms in the sun,
Colors paint the world with joy,
Nature's gift of love.


## Why LCEL?

As you can see, the legacy syntax has historically not been the easiest to learn. In addition to the `ConversationChain` object, LangChain has provided many other objects that perform more or less the same thing, introducing confusion on when to use what.

Additionally, what is not particularly clear by using the `ConversationChain` object, but there is a simple memory that is being kept each time that object is called. This may or may not be preferable given your use case, but in any regard, it is not ideal that this is abstracted away.

LCEL attempts to simplify the process by allowing users to chain LangChain objects together directly using the pipe (`|`) delimiter. There are many benefits to using LCEL, and you can read more about them [at this page](https://python.langchain.com/docs/expression_language/).

In [30]:
# Instantiating the OpenAI LLM
llm = ChatOpenAI(openai_api_key = API_KEYS['OPENAI_API_KEY'])

# Setting up the Chat prompt template
chat_prompt = ChatPromptTemplate.from_template('{input}')

In [31]:
# Instantiating the new chain with the LCEL syntax
new_conversation_chain = chat_prompt | llm

In [34]:
# Invoking the model appropriately
response = new_conversation_chain.invoke({'input': 'Write me a haiku about flowers.'})

print(response.content)

Petals soft and bright
Dancing in the gentle breeze
Nature's beauty glows


# Using MLX with LCEL
By default, there is currently no mechanism within the LangChain libraries that supports MLX; however, we still can make use of MLX in a custom capacity. In the next few cells, we will demonstrate how we can do this appropriately.

In [39]:
# Loading the quantized model and tokenizer with MLX
model, tokenizer = load(mlx_model_directory)

In [47]:
# Creating a class to represent our custom MLX LLM
class MLX_LLM(LLM):
    
    @property
    def _llm_type(self) -> str:
        return 'custom'
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None,):
        response = generate(
            model = model,
            tokenizer = tokenizer,
            prompt = prompt,
            max_tokens = 1000
        )
        
        return response

In [49]:
llm = MLX_LLM()

In [50]:
new_chain = chat_prompt | llm

In [51]:
# Invoking the model appropriately
response = new_chain.invoke({'input': 'Write me a haiku about flowers.'})



AttributeError: 'str' object has no attribute 'content'

In [52]:
response

"\n\nSure, here's a simple haiku about flowers:\n\nPetals bloom, soft breeze,\nWhispers life in gentle hue,\nSpring's sweet promise sings."